# Exploring US Wages Across Occupations



<img src="people-in-uniform-doing-different-occupations_1308-1835.jpg">




<img src="kagglescreenshot.png" width="400">
Others have already asked "IS there a wage gap?" and "How big is the wage gap?"  I wanted to ask a different question. I asked 




## Can we predict Occupation Category using wage, gender, share of females in the field, and if that field is male dominated or has a significant wage gap?

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import matplotlib.pyplot as plt
import itertools
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn import ensemble
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.svm import SVC

%matplotlib inline

In [3]:
import os
curr_directory = os.getcwd()
datafile = curr_directory + "/inc_occ_gender.csv"


In [4]:
genderwage = pd.read_csv(datafile)
genderwage.head()

,Occupation,All_workers,All_weekly,M_workers,M_weekly,F_workers,F_weekly
0,ALL OCCUPATIONS,109080,809,60746,895,48334,726
1,MANAGEMENT,12480,1351,7332,1486,5147,1139
2,Chief executives,1046,2041,763,2251,283,1836
3,General and operations managers,823,1260,621,1347,202,1002
4,Legislators,8,Na,5,Na,4,Na


In [5]:
genderwage.count()

Occupation     558
All_workers    558
All_weekly     558
M_workers      558
M_weekly       558
F_workers      558
F_weekly       558
dtype: int64

In [6]:
genderwage.dtypes

Occupation     object
All_workers     int64
All_weekly     object
M_workers       int64
M_weekly       object
F_workers       int64
F_weekly       object
dtype: object

In [7]:
genderwage[['All_weekly',
            'M_weekly',
            'F_weekly']] = genderwage[['All_weekly',
                                       'M_weekly',
                                       'F_weekly']].apply(pd.to_numeric, errors='coerce')

In [8]:
genderwage.dtypes

Occupation      object
All_workers      int64
All_weekly     float64
M_workers        int64
M_weekly       float64
F_workers        int64
F_weekly       float64
dtype: object

In [9]:
genderwage.isnull().sum()

Occupation       0
All_workers      0
All_weekly     236
M_workers        0
M_weekly       326
F_workers        0
F_weekly       366
dtype: int64

# First, we uncover the occupation categories and assign each row a label

In [10]:
categories = [] 
for i in genderwage['Occupation']:
    if i.isupper()==True:
        categories.append(i)

genderwage.insert(1, 'Occupation_Category', 0)

cat_count = -1
index_copy = pd.Index(list(genderwage['Occupation']))
for x in genderwage['Occupation']:
    current_index = index_copy.get_loc(x)
    genderwage.iloc[current_index,
                    genderwage.columns.get_loc('Occupation_Category')] = cat_count
    if x.isupper():
        cat_count = cat_count + 1
        genderwage.iloc[current_index,
                        genderwage.columns.get_loc('Occupation_Category')] = cat_count

genderwage.insert(1, 'Occupation_Category_label', 0)

cat_label = 'nothing'
for x in genderwage['Occupation']:
    current_index = index_copy.get_loc(x)
    genderwage.iloc[current_index,
                    genderwage.columns.get_loc('Occupation_Category_label')] = cat_label
    if x.isupper():
        cat_label = x
        genderwage.iloc[current_index,
                        genderwage.columns.get_loc('Occupation_Category_label')] = cat_label
    

In [11]:
categories = [] 
for i in genderwage['Occupation']:
    if i.isupper()==True:
        categories.append(i)

genderwage.insert(1, 'Occupation_Category', 0)

cat_count = -1
index_copy = pd.Index(list(genderwage['Occupation']))
for x in genderwage['Occupation']:
    current_index = index_copy.get_loc(x)
    genderwage.iloc[current_index,
                    genderwage.columns.get_loc('Occupation_Category')] = cat_count
    if x.isupper():
        cat_count = cat_count + 1
        genderwage.iloc[current_index,
                        genderwage.columns.get_loc('Occupation_Category')] = cat_count

genderwage.insert(1, 'Occupation_Category_label', 0)

cat_label = 'nothing'
for x in genderwage['Occupation']:
    current_index = index_copy.get_loc(x)
    genderwage.iloc[current_index,
                    genderwage.columns.get_loc('Occupation_Category_label')] = cat_label
    if x.isupper():
        cat_label = x
        genderwage.iloc[current_index,
                        genderwage.columns.get_loc('Occupation_Category_label')] = cat_label
    

ValueError: cannot insert Occupation_Category, already exists

In [ ]:
cleanedgenderwage = genderwage.dropna(axis=0, how='any').copy()
cleanedgenderwage = cleanedgenderwage.loc[~cleanedgenderwage['Occupation'].isin(categories)].reset_index(drop=True)


In [ ]:
cleanedgenderwage['Occupation_Category_label'].value_counts()

If we want to create a model that can work well with new information, we need to have categories that are not so narrow. To solve this problem, we're going to combine a few of them. Healthcare Support will be merged with Healthcare Professional, to become simply Healthcare. Agriculture, Construction, Maintenance, Transportation and Groundskeeping are all things we as a society need to continue, so they will all be rolled in to Infrastructure Support. Education will be rolled into Social Service, because both Education and Social Service jobs are focused on bettering people. Protective Service, Service, Arts, Legal, Engineering, and Science will all be put into Other because of their low count.

In [ ]:
# combine categories 
cleanedgenderwage.loc[cleanedgenderwage['Occupation_Category_label'] == 'HEALTHCARE SUPPORT', 'Occupation_Category_label'] = 'HEALTHCARE'
cleanedgenderwage.loc[cleanedgenderwage['Occupation_Category_label'] == 'HEALTHCARE PROFESSIONAL', 'Occupation_Category_label'] = 'HEALTHCARE'

cleanedgenderwage.loc[cleanedgenderwage['Occupation_Category_label'] == 'AGRICULTURAL', 'Occupation_Category_label'] = 'INFRASTRUCTURE SUPPORT'
cleanedgenderwage.loc[cleanedgenderwage['Occupation_Category_label'] == 'MAINTENANCE', 'Occupation_Category_label'] = 'INFRASTRUCTURE SUPPORT'
cleanedgenderwage.loc[cleanedgenderwage['Occupation_Category_label'] == 'GROUNDSKEEPING', 'Occupation_Category_label'] = 'INFRASTRUCTURE SUPPORT'
cleanedgenderwage.loc[cleanedgenderwage['Occupation_Category_label'] == 'TRANSPORTATION', 'Occupation_Category_label'] = 'INFRASTRUCTURE SUPPORT'

cleanedgenderwage.loc[cleanedgenderwage['Occupation_Category_label'] == 'EDUCATION', 'Occupation_Category_label'] = 'SOCIAL SERVICE'

cleanedgenderwage.loc[cleanedgenderwage['Occupation_Category_label'] == 'PROTECTIVE SERVICE', 'Occupation_Category_label'] = 'OTHER'
cleanedgenderwage.loc[cleanedgenderwage['Occupation_Category_label'] == 'SERVICE', 'Occupation_Category_label'] = 'OTHER'
cleanedgenderwage.loc[cleanedgenderwage['Occupation_Category_label'] == 'ARTS', 'Occupation_Category_label'] = 'OTHER'
cleanedgenderwage.loc[cleanedgenderwage['Occupation_Category_label'] == 'LEGAL', 'Occupation_Category_label'] = 'OTHER'
cleanedgenderwage.loc[cleanedgenderwage['Occupation_Category_label'] == 'ENGINEERING', 'Occupation_Category_label'] = 'OTHER'
cleanedgenderwage.loc[cleanedgenderwage['Occupation_Category_label'] == 'SCIENCE', 'Occupation_Category_label'] = 'OTHER'


In [ ]:
cleanedgenderwage['Occupation_Category_label'].value_counts()

# Distribution of Employees

In [ ]:
sns.kdeplot(data=cleanedgenderwage['F_workers'])
sns.kdeplot(data=cleanedgenderwage['M_workers'])
plt.xlabel('Number of employees')
plt.title('Distribution of Workers')
plt.show()

# Next the distribution of wages

In [ ]:
sns.kdeplot(data=cleanedgenderwage['F_weekly'])
sns.kdeplot(data=cleanedgenderwage['M_weekly'])
plt.title('Distribution of Wages')
plt.xlabel('Wages')
plt.show()

# Mens and Womens wages, compared to field wages

In [ ]:
# look at female wages vs male wages on a scatter plot
plt.plot(cleanedgenderwage['All_weekly'],
         cleanedgenderwage['F_weekly'], 'o', color='m')
plt.plot(cleanedgenderwage['All_weekly'],
         cleanedgenderwage['M_weekly'], 'o', color='c')
plt.xlabel('Occupation Median Salary')
plt.legend(['Females','Males'])
plt.title('Male Vs Female Wages')
plt.ylabel('Salary')
plt.show()

## Next I marked which occupations are male dominated

Any ratio under .45 is considered male dominated as that shows a distinct majority over women.

## I also made a column marking occupations with significant wage gaps

For our purposes, a significant wage gap is a difference of 15% or more over women.

In [ ]:
cleanedgenderwage['WageGap_as_Ratio'] = cleanedgenderwage['F_weekly'] / cleanedgenderwage['M_weekly']
cleanedgenderwage['Significant_wage_gap'] = np.where(cleanedgenderwage['WageGap_as_Ratio'] > .85, 0, 1)
cleanedgenderwage['F_share'] = cleanedgenderwage['F_workers'] / cleanedgenderwage['All_workers']
cleanedgenderwage['Male_Dominated'] = np.where(cleanedgenderwage['F_share'] > .45, 0, 1)


# Kernel Density Estimate Plot
# for the Wage Ratio

In [ ]:
sns.kdeplot(data=cleanedgenderwage['WageGap_as_Ratio'])
plt.show()

# Female Share per Occupation

In [ ]:

# Now lets look at the ratio of workers
plt.figure(figsize=((10,30)))
sns.barplot(x='F_share',
            y='Occupation',
            data=cleanedgenderwage.sort_values('F_share', ascending=False))
plt.axvline(x=.5, color='black', linestyle='--') # equality at 50%
plt.show()

# Wage Ratio By Occupation

In [ ]:
# Look at wage ratio by occupation
plt.figure(figsize=((10,30)))
sns.barplot(x='WageGap_as_Ratio',
            y='Occupation',
            data=cleanedgenderwage.sort_values('WageGap_as_Ratio',
                                               ascending=False))
plt.axvline(x=1, color='black', linestyle='solid')
plt.title('Wage Ratio by Occupation')
plt.show()

# Time for some MODELS


The question I'm asking is a classification question, so I tested:
    * Logistic Regression Classifier  (A Classic first step classifier)
    * Decision Tree Classifier (a different kind of classifier than a regression)
    * Random Forest Classifier (multiple decision trees together as a classifier)
    * Extra Trees Ensemble Classifier (more trees might be better)
    * Support Vector Classifier (a third kind of classifier)
    * Gradient Boosting Classifier (a  fourth kind of classifier)


In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(k_neighbors=3, ratio='auto', kind='regular')

## Assign our variables, scale our X, use SMOTE to expand our training data

In [ ]:
scaler = StandardScaler()
columns = ['M_weekly',
           'F_weekly',
           'Male_Dominated',
           'Significant_wage_gap',
           'WageGap_as_Ratio',
           'F_share']

X = cleanedgenderwage[columns]

X_std = StandardScaler().fit_transform(X)
y = cleanedgenderwage['Occupation_Category_label']

X_train, X_test, y_train, y_test = train_test_split(X_std,
                                                    y,
                                                    test_size=.4,
                                                    random_state=64)
# using random state so I can rerun cells 
# and not accidentally change the number of instances in my classes

X_smote, y_smote = smote.fit_sample(X_train, y_train)

In [ ]:
y_test.value_counts()

## I wrote a small function so that I could use GridSearchCV to check multiple estimators without having to manually set each one up. 

In [ ]:
from sklearn.model_selection import GridSearchCV

def check_grid(estimator, params, X, y):
    grid_classifier = GridSearchCV(estimator=estimator, param_grid=params)
    grid_classifier.fit(X, y)
    grid_score = grid_classifier.score(X, y)
    return grid_score

## Create the dictionary of each estimators parameters to check, set up the different estimators, set up list of estimators and list of params.

In [ ]:
log_param = {'tol': (.01, .5, 1, 5),
             'C': (.01, .5, 1, 5),
             'solver': ('newton-cg', 'lbfgs', 'sag')}
forest_type_param = {'n_estimators': (100, 200, 300),
                       'criterion': ('gini', 'entropy'),
                       'max_depth': (2, 3, 5)}
dtree_param = {'criterion': ('gini', 'entropy'),
                       'max_depth': (2, 3, 5)}
gradient_param = {'learning_rate': (.01, .5, 1),
                   'n_estimators':(100, 200, 300),
                   'max_depth':(2, 3, 4, 5, 6)}
svc_param = {'C': (.1, .5, 1, 5),
             'kernel': ('linear', 'poly', 'rbf', 'sigmoid'),
             'degree': (2, 3, 4, 5),
             'gamma': (1, 2, 3)}

# got my estimators
logr = linear_model.LogisticRegression()
gradient = ensemble.GradientBoostingClassifier()
extree = ensemble.ExtraTreesClassifier()
dtree = tree.DecisionTreeClassifier()
svc = SVC()
forest = ensemble.RandomForestClassifier()

estimators = [logr,
              gradient,
              extree,
              dtree,
              svc,
              forest]
params = [log_param,
          gradient_param,
          forest_type_param,
          dtree_param,
          svc_param,
          forest_type_param]


## Iterate over both lists, using my function to check each estimator and its corresponding list of parameters.

In [ ]:

for estimator, param in list(zip(estimators, params)):
    results = []
    results.append(estimator)
    results.append(check_grid(estimator, param, X_smote, y_smote))
    print(results)



## validating our  model with the test set

In [ ]:
for estimator, param in list(zip(estimators, params)):
    results = []
    results.append(estimator)
    results.append(check_grid(estimator, param, X_test, y_test))
    print(results)


# The confusion matrix of our best scoring model

In [ ]:
# look at confusion matrix here
y_pred = gradient.fit(X_smote, y_smote).predict(X_test)
c_matrix = metrics.confusion_matrix(y_test, y_pred)

class_names = cleanedgenderwage['Occupation_Category_label'].unique();

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Reds):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    # print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


plt.figure(figsize=(10, 10))
plot_confusion_matrix(c_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')
plt.show()

# Conclusions

Random chance gives each of our 11 labels an equally likely 9% chance if we just guess. The poorest performing model still returned a score above 45%. 

# Next steps go here